# Naloga razvrščanja besedila

V tem modulu bomo začeli s preprosto nalogo razvrščanja besedila na podlagi nabora podatkov **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: razvrstili bomo naslove novic v eno od 4 kategorij: Svet, Šport, Posel in Znanost/Tehnologija.

## Nabor podatkov

Za nalaganje nabora podatkov bomo uporabili API **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Zdaj lahko dostopamo do učnih in testnih delov nabora podatkov z uporabo `dataset['train']` in `dataset['test']` oziroma:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Natisnimo prvih 10 novih naslovov iz našega nabora podatkov:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Vektorizacija besedila

Zdaj moramo besedilo pretvoriti v **številke**, ki jih lahko predstavimo kot tenzorje. Če želimo predstavitev na ravni besed, moramo narediti dve stvari:

* Uporabiti **tokenizator**, da razdelimo besedilo na **tokene**.
* Zgraditi **besedišče** teh tokenov.

### Omejevanje velikosti besedišča

V primeru podatkovnega nabora AG News je velikost besedišča precej velika, več kot 100.000 besed. Na splošno ne potrebujemo besed, ki se redko pojavljajo v besedilu — le nekaj stavkov jih bo vsebovalo, in model se iz njih ne bo učil. Zato je smiselno omejiti velikost besedišča na manjše število z dodajanjem argumenta konstruktorju vektorizatorja:

Oba koraka lahko izvedemo z uporabo sloja **TextVectorization**. Ustvarimo objekt vektorizatorja in nato pokličimo metodo `adapt`, da preletimo vse besedilo in zgradimo besedišče:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Opomba** uporabljamo le podmnožico celotnega nabora podatkov za gradnjo besedišča. To počnemo, da pospešimo čas izvajanja in vam ni treba čakati. Vendar pa s tem tvegamo, da nekaterih besed iz celotnega nabora podatkov ne bo vključenih v besedišče in bodo med usposabljanjem prezrte. Zato bi uporaba celotne velikosti besedišča in obdelava celotnega nabora podatkov med `adapt` lahko povečala končno natančnost, vendar ne bistveno.

Zdaj lahko dostopamo do dejanskega besedišča:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Z uporabo vektorizatorja lahko enostavno kodiramo katero koli besedilo v niz številk:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Predstavitev besedila z metodo vreče besed

Ker besede predstavljajo pomen, lahko včasih ugotovimo pomen določenega besedila že samo z opazovanjem posameznih besed, ne glede na njihov vrstni red v stavku. Na primer, pri razvrščanju novic besede, kot sta *vreme* in *sneg*, verjetno nakazujejo na *vremensko napoved*, medtem ko bi besede, kot sta *delnice* in *dolar*, kazale na *finančne novice*.

**Predstavitev z metodo vreče besed** (BoW) je najpreprostejša za razumevanje med tradicionalnimi vektorskimi predstavitvami. Vsaka beseda je povezana z indeksom vektorja, element vektorja pa vsebuje število pojavitev posamezne besede v določenem dokumentu.

![Slika, ki prikazuje, kako je predstavitev z metodo vreče besed predstavljena v pomnilniku.](../../../../../translated_images/sl/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Opomba**: Na metodo BoW lahko gledate tudi kot na vsoto vseh eno-vroče kodiranih vektorjev za posamezne besede v besedilu.

Spodaj je primer, kako ustvariti predstavitev z metodo vreče besed z uporabo knjižnice Scikit Learn v Pythonu:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Prav tako lahko uporabimo Kerasov vektorizator, ki smo ga definirali zgoraj, vsako številko besede pretvorimo v eno-vroče kodiranje in vse te vektorje seštejemo:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Opomba**: Morda vas bo presenetilo, da se rezultat razlikuje od prejšnjega primera. Razlog za to je, da v primeru s Keras dolžina vektorja ustreza velikosti besedišča, ki je bilo ustvarjeno iz celotnega nabora podatkov AG News, medtem ko smo v primeru s Scikit Learn besedišče ustvarili sproti iz vzorčnega besedila.


## Učenje klasifikatorja BoW

Zdaj, ko smo se naučili, kako zgraditi predstavitev besedne vreče za našo besedilo, lahko začnemo z učenjem klasifikatorja, ki jo uporablja. Najprej moramo naš nabor podatkov pretvoriti v predstavitev besedne vreče. To lahko dosežemo z uporabo funkcije `map` na naslednji način:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Zdaj definirajmo preprosto nevronsko mrežo za klasifikacijo, ki vsebuje eno linearno plast. Velikost vhodnega sloja je `vocab_size`, velikost izhodnega sloja pa ustreza številu razredov (4). Ker rešujemo nalogo klasifikacije, je končna aktivacijska funkcija **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Ker imamo 4 razrede, je natančnost nad 80 % dober rezultat.

## Učenje klasifikatorja kot ene mreže

Ker je vektorizator prav tako Kerasov sloj, lahko definiramo mrežo, ki ga vključuje, in jo učimo od začetka do konca. Na ta način nam ni treba vektorizirati nabora podatkov z uporabo `map`, temveč lahko prvotni nabor podatkov preprosto posredujemo vhodu mreže.

> **Opomba**: Še vedno bi morali uporabiti `map` na našem naboru podatkov, da bi polja iz slovarjev (kot so `title`, `description` in `label`) pretvorili v torke. Vendar pa lahko ob nalaganju podatkov z diska že na začetku ustvarimo nabor podatkov z zahtevano strukturo.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrami, trigrami in n-grami

Ena od omejitev pristopa vreče besed je, da so nekatere besede del večbesednih izrazov. Na primer, beseda 'hot dog' ima popolnoma drugačen pomen kot besedi 'hot' in 'dog' v drugih kontekstih. Če besedi 'hot' in 'dog' vedno predstavljamo z istimi vektorji, lahko to zmede naš model.

Da bi to rešili, se pogosto uporabljajo **n-gramske reprezentacije** pri metodah klasifikacije dokumentov, kjer je frekvenca vsake besede, dvobesedne ali tribesedne kombinacije koristna značilnost za učenje klasifikatorjev. Pri bigramskih reprezentacijah, na primer, dodamo vse pare besed v besedišče, poleg originalnih besed.

Spodaj je primer, kako ustvariti bigramsko vrečo besed z uporabo Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Glavna pomanjkljivost pristopa n-gramov je, da se velikost besedišča začne izjemno hitro povečevati. V praksi moramo združiti predstavitev n-gramov s tehniko zmanjševanja dimenzionalnosti, kot so *vgnezditve* (embeddings), o katerih bomo govorili v naslednji enoti.

Da uporabimo predstavitev n-gramov v našem **AG News** podatkovnem naboru, moramo parameter `ngrams` posredovati konstruktorju `TextVectorization`. Dolžina besedišča bigramov je **znatno večja**, v našem primeru več kot 1,3 milijona tokenov! Zato je smiselno omejiti tudi število bigramov na neko razumno vrednost.

Lahko bi uporabili isto kodo kot zgoraj za treniranje klasifikatorja, vendar bi bila to zelo neučinkovita uporaba pomnilnika. V naslednji enoti bomo trenirali klasifikator bigramov z uporabo vgnezditve. Medtem pa lahko eksperimentirate s treniranjem klasifikatorja bigramov v tej beležnici in preverite, ali lahko dosežete višjo natančnost.


## Samodejno izračunavanje BoW vektorjev

V zgornjem primeru smo BoW vektorje izračunali ročno, tako da smo sešteli enovrstne kodiranja posameznih besed. Vendar pa nam najnovejša različica TensorFlow omogoča samodejno izračunavanje BoW vektorjev, če parameter `output_mode='count` posredujemo konstruktorju vektorizatorja. To bistveno poenostavi definiranje in treniranje našega modela:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Frekvenca izraza - obratna frekvenca dokumenta (TF-IDF)

V predstavitvi BoW so pojavitve besed utežene z enako tehniko, ne glede na samo besedo. Vendar je jasno, da so pogoste besede, kot sta *a* in *in*, veliko manj pomembne za klasifikacijo kot specializirani izrazi. Pri večini nalog obdelave naravnega jezika (NLP) so nekatere besede bolj relevantne kot druge.

**TF-IDF** pomeni **frekvenca izraza - obratna frekvenca dokumenta**. Gre za različico metode bag-of-words, kjer namesto binarne vrednosti 0/1, ki označuje pojav besede v dokumentu, uporabimo številsko vrednost s plavajočo vejico, ki je povezana s frekvenco pojavljanja besede v korpusu.

Bolj formalno je utež $w_{ij}$ besede $i$ v dokumentu $j$ definirana kot:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kjer
* $tf_{ij}$ predstavlja število pojavitev $i$ v $j$, torej vrednost BoW, ki smo jo videli prej
* $N$ je število dokumentov v zbirki
* $df_i$ je število dokumentov, ki vsebujejo besedo $i$ v celotni zbirki

Vrednost TF-IDF $w_{ij}$ se povečuje sorazmerno s številom pojavitev besede v dokumentu in se zmanjša glede na število dokumentov v korpusu, ki vsebujejo to besedo. To pomaga prilagoditi dejstvo, da se nekatere besede pojavljajo pogosteje kot druge. Na primer, če se beseda pojavi v *vsakem* dokumentu v zbirki, potem velja $df_i=N$, in $w_{ij}=0$, kar pomeni, da so ti izrazi popolnoma zanemarjeni.

TF-IDF vektorizacijo besedila lahko preprosto ustvarite s pomočjo Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

V Kerasu lahko plast `TextVectorization` samodejno izračuna TF-IDF frekvence z uporabo parametra `output_mode='tf-idf'`. Ponovimo kodo, ki smo jo uporabili zgoraj, da vidimo, ali uporaba TF-IDF poveča natančnost:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Zaključek

Čeprav TF-IDF predstavitve dodeljujejo uteži pogostosti različnim besedam, ne morejo predstaviti pomena ali vrstnega reda. Kot je slavni jezikoslovec J. R. Firth leta 1935 dejal: "Popoln pomen besede je vedno kontekstualen, in nobena študija pomena brez konteksta ne more biti resna." Kasneje v tečaju se bomo naučili, kako zajeti kontekstualne informacije iz besedila z uporabo jezikovnega modeliranja.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
